<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_2D-CNN/blob/main/TimeSeries_2DCNN_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun,JANG
# limitsinx.tistory.com
# Last revision date : 2023.04.29

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, GRU, Dropout, BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model


xy_train = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/train_v.txt',delimiter=',',dtype=np.float32) #5808
xy_test = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/test_v.txt',delimiter=',',dtype=np.float32) #10883
xy_train = np.delete(xy_train ,3, axis = 1)
xy_test = np.delete(xy_test ,3, axis = 1)

start_test = 1000
middle_test = 2000

middle_test1 = 3000
end_test = 10000

xy_test_1 = xy_test[:start_test,:]
xy_test_1n2 = xy_test[start_test:middle_test,:]
xy_test_2 = xy_test[middle_test1:end_test,:]

xy_train = np.concatenate((xy_train, xy_test_1, xy_test_2), axis = 0)
xy_test = xy_test_1n2

print("Train : ", xy_train.shape)
print("Test : " , xy_test.shape)

def MinMaxScaler(data) : # Normal Std
  numerator = data - np.min(data,0)
  denominator = np.max(data,0) - np.min(data,0)

  return numerator / (denominator + 1e-7)

def origin_minmax(data) : # HJ STYLE
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])

  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max_train, arr_min_train = origin_minmax(xy_train)
arr_max_test, arr_min_test = origin_minmax(xy_test)
arr_max_total = arr_max_train
arr_min_total = arr_min_train
print(arr_max_train, arr_min_train)
print(arr_max_test, arr_min_test)

for i in range(xy_train.shape[1]) : 
  if(arr_max_train[i] > arr_max_test[i]) : 
    arr_max_total[i] = arr_max_train[i]
  else : 
    arr_max_total[i] = arr_max_test[i]
  pass

for i in range(xy_train.shape[1]) : 
  if(arr_min_train[i] < arr_min_test[i]) : 
    arr_min_total[i] = arr_min_train[i]
  else : 
    arr_min_total[i] = arr_min_test[i]
  pass

print(arr_max_total, arr_min_total)

def HJ_MinMaxScaler(data) : 
  for i in range(data.shape[1]) : 
    numerator = data[:,i] - arr_min_total[i]
    denominator = arr_max_total[i] - arr_min_total[i]
    data[:,i] = numerator / (denominator + 1e-7)
    
  return data

def HJ_backMinMax(data) : 
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (arr_max_total[i] - arr_min_total[i]) + arr_min_total[i]

  return data

xy_train_scale = HJ_MinMaxScaler(xy_train)
xy_test_scale = HJ_MinMaxScaler(xy_test)



In [ ]:
print(xy_train.shape)
print(xy_test.shape)

In [ ]:
def reshape_timeseries_to_image(data, window_size):
    num_features = data.shape[1]
    num_samples = data.shape[0] - window_size + 1
    reshaped_data = np.zeros((num_samples, window_size, num_features))

    for i in range(num_samples):
        reshaped_data[i] = data[i:i+window_size]

    return reshaped_data

In [ ]:
TIME_STEPS = 6

train_image = reshape_timeseries_to_image(xy_train, TIME_STEPS)
test_image = reshape_timeseries_to_image(xy_test, TIME_STEPS)
print("Train image.shape : ", train_image.shape,train_image[0,:,:])
print("Test image.shape : " ,test_image.shape, test_image[0,:,:])

In [ ]:
input_shape = (TIME_STEPS, train_image.shape[2], 1)
inputs = Input(shape=input_shape)

# Encoding
x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 1), padding='same')(x)

# Decoding
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 1))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# 학습
autoencoder.fit(train_image, train_image, epochs=50, batch_size=32)

In [ ]:
# Plotting
original_images = np.copy(train_image)
trainY_back_1 = original_images.reshape(-1, original_images.shape[-1])
trainY_back = HJ_backMinMax(trainY_back_1)

decoded_images_input = np.copy(train_image)
decoded_images = autoencoder.predict(decoded_images_input)
print("Original Decoded : ", decoded_images)

decoded_images = np.squeeze(decoded_images, axis = 3)
trainY_predict_2 = decoded_images.reshape(-1, decoded_images.shape[-1])
trainY_predict_back = HJ_backMinMax(trainY_predict_2)
print("Decoded : ", trainY_predict_back.shape)


In [ ]:
print("Original : ", trainY_back)
print("Predicted : " ,trainY_predict_back)

In [ ]:
#Plotting Training data
seq_length = 0
plt.figure(figsize = (10,5))
plt.subplot(2,2,1)
plt.plot(trainY_back[seq_length:,0], color = 'blue')
plt.plot(trainY_predict_back[:,0], color = 'red')
plt.legend(['Real Current','Predicted Current'])

plt.subplot(2,2,2)
plt.plot(trainY_back[seq_length:,1], color = 'blue')
plt.plot(trainY_predict_back[:,1], color = 'red')
plt.legend(['Real Temperature','Predicted Temperature'])

plt.subplot(2,2,3)
plt.plot(trainY_back[seq_length:,2], color = 'blue')
plt.plot(trainY_predict_back[:,2], color = 'red')
plt.legend(['Real SOC','Predicted SOC'])

plt.subplot(2,2,4)
plt.plot(trainY_back[seq_length:,3], color = 'blue')
plt.plot(trainY_predict_back[:,3], color = 'red')
plt.legend(['Real Voltage','Predicted Voltage'])

error_current_1 = trainY_back[:,0] - trainY_predict_back[:,0]
error_temp_1 = trainY_back[:,1] - trainY_predict_back[:,1]
error_soc_1 = trainY_back[:,2] - trainY_predict_back[:,2]
error_voltage_1 = trainY_back[:,3] - trainY_predict_back[:,3]

plt.figure(figsize = (10,5))
plt.subplot(2,2,1)
plt.plot(error_current_1, color = 'green')
plt.legend(['Current Error'])
plt.ylim(-10, 10)

plt.subplot(2,2,2)
plt.plot(error_temp_1, color = 'green')
plt.legend(['Temperature Error'])
plt.ylim(-2, 2)

plt.subplot(2,2,3)
plt.plot(error_soc_1, color = 'green')
plt.legend(['SOC Error'])
plt.ylim(-2, 2)

plt.subplot(2,2,4)
plt.plot(error_voltage_1, color = 'green')
plt.legend(['Voltage Error'])
plt.ylim(-10, 10)


In [ ]:
test_final_output = autoencoder.predict(test_image)
print("Final Output : ", test_final_output.shape)

In [ ]:
# Testing
testY_predict = np.copy(test_final_output)
print(testY_predict.shape)
testY_predict_1 = np.squeeze(testY_predict, axis = 3)
testY_predict_2 = testY_predict_1.reshape(-1, testY_predict_1.shape[-1])
testY_predict_back = HJ_backMinMax(testY_predict_2)

test_image_input = np.copy(test_image)
testY_back_1 = test_image_input.reshape(-1, test_image_input.shape[-1])
testY_back = HJ_backMinMax(testY_back_1)

print(testY_predict_back.shape)
print(testY_back.shape)

In [ ]:
#Plotting Test Data
import matplotlib.pyplot as plt

seq_length = 0

plt.figure(figsize = (10,5))
plt.subplot(2,2,1)
plt.plot(testY_back[seq_length:,0], color = 'blue')
plt.plot(testY_predict_back[:,0], color = 'red')
plt.legend(['Real Current','Predicted Current'])

plt.subplot(2,2,2)
plt.plot(testY_back[seq_length:,1], color = 'blue')
plt.plot(testY_predict_back[:,1], color = 'red')
plt.legend(['Real Temperature','Predicted Temperature'])

plt.subplot(2,2,3)
plt.plot(testY_back[seq_length:,2], color = 'blue')
plt.plot(testY_predict_back[:,2], color = 'red')
plt.legend(['Real SOC','Predicted SOC'])

plt.subplot(2,2,4)
plt.plot(testY_back[seq_length:,3], color = 'blue')
plt.plot(testY_predict_back[:,3], color = 'red')
plt.legend(['Real Voltage','Predicted Voltage'])


error_current_1 = testY_back[:,0] - testY_predict_back[:,0]
error_temp_1 = testY_back[:,1] - testY_predict_back[:,1]
error_soc_1 = testY_back[:,2] - testY_predict_back[:,2]
error_voltage_1 = testY_back[:,3] - testY_predict_back[:,3]

plt.figure(figsize = (10,5))
plt.subplot(2,2,1)
plt.plot(error_current_1, color = 'green')
plt.legend(['Current Error'])
plt.ylim(-10, 10)

plt.subplot(2,2,2)
plt.plot(error_temp_1, color = 'green')
plt.legend(['Temperature Error'])
plt.ylim(-1, 1)

plt.subplot(2,2,3)
plt.plot(error_soc_1, color = 'green')
plt.legend(['SOC Error'])
plt.ylim(-2, 2)

plt.subplot(2,2,4)
plt.plot(error_voltage_1, color = 'green')
plt.legend(['Voltage Error'])
plt.ylim(-10, 10)